## **2.3 学习正则表达式 re**

### 2.3.1 为什么使用正则表达式？
典型的搜索和替换操作要求您提供与预期的搜索结果匹配的确切文本。虽然这种技术对于对静态文本执行简单搜索和替换任务可能已经足够了，但它缺乏灵活性，若采用这种方法搜索动态文本，即使不是不可能，至少也会变得很困难。

通过使用正则表达式，可以：

    - 测试字符串内的模式。
        例如，可以测试输入字符串，以查看字符串内是否出现电话号码模式或信用卡号码模式。这称为数据验证。
    - 替换文本。
        可以使用正则表达式来识别文档中的特定文本，完全删除该文本或者用其他文本替换它。
    - 基于模式匹配从字符串中提取子字符串。
        可以查找文档内或输入域内特定的文本。
可以使用正则表达式来搜索和替换标记。

#### 使用正则表达式的优势是什么？  **简洁**
- 正则表达式是用来简洁表达一组字符串的表达式
- 正则表达式是一种通用的字符串表达框架
- 正则表达式是一种针对字符串表达“简洁”和“特征”思想的工具
- 正则表达式可以用来判断某字符串的特征归属

#### 正则表达式在文本处理中十分常用：
- 同时查找或替换一组字符串
- 匹配字符串的全部或部分(主要)

### 2.3.2 正则表达式语法

正则表达式语法由字符和操作符构成:

- 常用操作符
    - `.` 表示任何单个字符
    - `[ ]` 字符集，对单个字符给出取值范围 ，如`[abc]`表示a、b、c，`[a‐z]`表示a到z单个字符
    - `[^ ]` 非字符集，对单个字符给出排除范围 ，如`[^abc]`表示非a或b或c的单个字符
    - `*` 前一个字符0次或无限次扩展，如abc* 表示 ab、abc、abcc、abccc等 
    - `+` 前一个字符1次或无限次扩展 ，如abc+ 表示 abc、abcc、abccc等 
    - `?` 前一个字符0次或1次扩展 ，如abc? 表示 ab、abc
    - `|` 左右表达式任意一个 ，如abc|def 表示 abc、def

    - `{m}` 扩展前一个字符m次 ，如ab{2}c表示abbc
    - `{m,n}` 扩展前一个字符m至n次（含n） ，如ab{1,2}c表示abc、abbc
    - `^` 匹配字符串开头 ，如^abc表示abc且在一个字符串的开头
    - `$` 匹配字符串结尾 ，如abc$表示abc且在一个字符串的结尾
    - `( )` 分组标记，内部只能使用 | 操作符 ，如(abc)表示abc，(abc|def)表示abc、def
    - `\d` 数字，等价于`[0‐9]`
    - `\w` 单词字符，等价于`[A‐Za‐z0‐9_]`

### 2.3.3 正则表达式re库的使用

- 调用方式：import re
- re库采用raw string类型表示正则表达式，表示为：r'text'，raw string是不包含对转义符再次转义的字符串;

#### re库的主要功能函数：

- re.search() 在一个字符串中搜索匹配正则表达式的第一个位置，返回match对象 
    - re.search(pattern, string, flags=0)
- re.match() 从一个字符串的开始位置起匹配正则表达式，返回match对象
    - re.match(pattern, string, flags=0)
- re.findall() 搜索字符串，以列表类型返回全部能匹配的子串
    - re.findall(pattern, string, flags=0)
- re.split() 将一个字符串按照正则表达式匹配结果进行分割，返回列表类型
    - re.split(pattern, string, maxsplit=0, flags=0)
- re.finditer() 搜索字符串，返回一个匹配结果的迭代类型，每个迭代元素是match对象
    - re.finditer(pattern, string, flags=0)
- re.sub() 在一个字符串中替换所有匹配正则表达式的子串，返回替换后的字符串
    - re.sub(pattern, repl, string, count=0, flags=0)

    -  flags : 正则表达式使用时的控制标记：
        - re.I -->  re.IGNORECASE :  忽略正则表达式的大小写，`[A‐Z]`能够匹配小写字符
        - re.M -->  re.MULTILINE :  正则表达式中的^操作符能够将给定字符串的每行当作匹配开始
        - re.S -->  re.DOTALL   :  正则表达式中的.操作符能够匹配所有字符，默认匹配除换行外的所有字符
        
#### re库的另一种等价用法（编译）

- regex = re.compile(pattern, flags=0)：将正则表达式的字符串形式编译成正则表达式对象

#### re 库的贪婪匹配和最小匹配

- `.*` Re库默认采用贪婪匹配，即输出匹配最长的子串
- `*?` 只要长度输出可能不同的，都可以通过在操作符后增加?变成最小匹配

### 2.3.4 实战：淘宝商品比价定向爬虫

- 爬取网址：https://s.taobao.com/search?q=书包&js=1&stats_click=search_radio_all%25
- 爬取思路：
    1. 提交商品搜索请求，循环获取页面
    2. 对于每个页面，提取商品名称和价格信息
    3. 将信息输出到屏幕上

In [1]:
# 导入包
import requests
import re

#### 1. 提交商品搜索请求，循环获取页面

In [2]:
def getHTMLText(url):
    """
    请求获取html，（字符串）
    :param url: 爬取网址
    :return: 字符串
    """
    try:
        # 添加头信息,
        kv = {
            'cookie': 'thw=cn; v=0; t=ab66dffdedcb481f77fd563809639584; cookie2=1f14e41c704ef58f8b66ff509d0d122e; _tb_token_=5e6bed8635536; cna=fGOnFZvieDECAXWIVi96eKju; unb=1864721683; sg=%E4%B8%8B3f; _l_g_=Ug%3D%3D; skt=83871ef3b7a49a0f; cookie1=BqeGegkL%2BLUif2jpoUcc6t6Ogy0RFtJuYXR4VHB7W0A%3D; csg=3f233d33; uc3=vt3=F8dBy3%2F50cpZbAursCI%3D&id2=UondEBnuqeCnfA%3D%3D&nk2=u%2F5wdRaOPk21wDx%2F&lg2=VFC%2FuZ9ayeYq2g%3D%3D; existShop=MTU2MjUyMzkyMw%3D%3D; tracknick=%5Cu4E36%5Cu541B%5Cu4E34%5Cu4E3F%5Cu5929%5Cu4E0B; lgc=%5Cu4E36%5Cu541B%5Cu4E34%5Cu4E3F%5Cu5929%5Cu4E0B; _cc_=WqG3DMC9EA%3D%3D; dnk=%5Cu4E36%5Cu541B%5Cu4E34%5Cu4E3F%5Cu5929%5Cu4E0B; _nk_=%5Cu4E36%5Cu541B%5Cu4E34%5Cu4E3F%5Cu5929%5Cu4E0B; cookie17=UondEBnuqeCnfA%3D%3D; tg=0; enc=2GbbFv3joWCJmxVZNFLPuxUUDA7QTpES2D5NF0D6T1EIvSUqKbx15CNrsn7nR9g%2Fz8gPUYbZEI95bhHG8M9pwA%3D%3D; hng=CN%7Czh-CN%7CCNY%7C156; mt=ci=32_1; alitrackid=www.taobao.com; lastalitrackid=www.taobao.com; swfstore=97213; x=e%3D1%26p%3D*%26s%3D0%26c%3D0%26f%3D0%26g%3D0%26t%3D0%26__ll%3D-1%26_ato%3D0; uc1=cookie16=UtASsssmPlP%2Ff1IHDsDaPRu%2BPw%3D%3D&cookie21=UIHiLt3xThH8t7YQouiW&cookie15=URm48syIIVrSKA%3D%3D&existShop=false&pas=0&cookie14=UoTaGqj%2FcX1yKw%3D%3D&tag=8&lng=zh_CN; JSESSIONID=A502D8EDDCE7B58F15F170380A767027; isg=BMnJJFqj8FrUHowu4yKyNXcd2PXjvpa98f4aQWs-RbDvsunEs2bNGLfj8BYE6lWA; l=cBTDZx2mqxnxDRr0BOCanurza77OSIRYYuPzaNbMi_5dd6T114_OkmrjfF96VjWdO2LB4G2npwJ9-etkZ1QoqpJRWkvP.; whl=-1%260%260%261562528831082',
            'user-agent': 'Mozilla/5.0'
        }
        r = requests.get(url, timeout=30, headers=kv)
        # r = requests.get(url, timeout=30)
        # print(r.status_code)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "爬取失败"


#### 2. 对于每个页面，提取商品名称和价格信息

In [3]:
def parsePage(glist, html):
    '''
    解析网页，搜索需要的信息
    :param glist: 列表作为存储容器
    :param html: 由getHTMLText()得到的
    :return: 商品信息的列表
    '''
    try:
        # 使用正则表达式提取信息
        price_list = re.findall(r'\"view_price\"\:\"[\d\.]*\"', html)
        name_list = re.findall(r'\"raw_title\"\:\".*?\"', html)
        for i in range(len(price_list)):
            price = eval(price_list[i].split(":")[1])  #eval（）在此可以去掉""
            name = eval(name_list[i].split(":")[1])
            glist.append([price, name])
    except:
        print("解析失败")

#### 3. 将信息输出到屏幕上

In [4]:
def printGoodList(glist):
    tplt = "{0:^4}\t{1:^6}\t{2:^10}"
    print(tplt.format("序号", "商品价格", "商品名称"))
    count = 0
    for g in glist:
        count = count + 1
        print(tplt.format(count, g[0], g[1]))

In [5]:
# 根据页面url的变化寻找规律，构建爬取url
goods_name = "书包"  # 搜索商品类型
start_url = "https://s.taobao.com/search?q=" + goods_name
info_list = []
page = 3  # 爬取页面数量

In [6]:
count = 0
for i in range(page):
    count += 1
    try:
        url = start_url + "&s=" + str(44 * i)
        html = getHTMLText(url)  # 爬取url
        parsePage(info_list, html) #解析HTML和爬取内容
        print("\r爬取页面当前进度: {:.2f}%".format(count * 100 / page), end="")  # 显示进度条
    except:
        continue

爬取页面当前进度: 100.00%

In [7]:
printGoodList(info_list)

 序号 	 商品价格 	   商品名称   
 1  	139.00	卡拉羊双肩包书包男女潮流中学生书包初中生
 2  	149.00	鳄鱼男士双肩包商务休闲电脑帆布背包旅游旅行包时尚潮流学生书包
 3  	59.00 	电脑背包男士双肩包旅行大容量时尚潮流高中初中学生书包女大学生
 4  	79.90 	背包男士双肩包时尚潮流旅行包轻便旅游电脑包休闲高中大学生书包
 5  	69.90 	HK双肩包男简约书包时尚潮流休闲电脑包旅行轻便高中生大学生背包
 6  	219.00	NIKE耐克双肩包男包女包官网旗舰高中初中运动包学生书包旅行背包
 7  	62.90 	书包男韩版原宿ulzzang高中双肩包女中学生大容量初中生背包ins潮
 8  	279.00	简佰格2020年新款春书包森系束口抽绳包女双肩背包休闲文艺小清新
 9  	279.00	Jordan 官方 AIR JORDAN 双肩包 书包背包
 CW7699
 10 	248.00	简佰格双肩包女2020新款时尚韩版书包ins风学生包包百搭旅游背包
 11 	149.00	阿迪达斯双肩包男包女包大容量户外休闲运动包学生书包登山包背包
 12 	119.00	mracehomme2020新款韩版潮流学生书包女时尚拼色百搭双肩包男背包
 13 	89.00 	背包男士简约时尚街头潮流个性书包休闲轻便电脑包旅行户外双肩包
 14 	899.00	【买1送1】Fion/菲安妮大牌双肩包女 2020新款印花背包旅行书包
 15 	329.00	新款双肩包女韩版学生初高中书包大运动包校园电脑背包男潮甜甜圈
 16 	1098.00	Herschel Dawson大号时尚双肩包 Surplus系列休闲背包书包10649
 17 	748.00	Herschel x Coca Cola联名款双肩包 Nova中号背包 学生书包10503
 18 	288.00	【新品】JanSport杰斯伯双肩包女学生书包电脑包休闲背包4QUT5L8
 19 	439.00	kipling男女大容量电脑背包2020年新款旅行包书包双肩包|BACKPACK
 20 	69.00 	牛津布双肩包女2020新款潮韩版时尚百搭大学生书包旅行帆布小背包
 21 	79.00 	电脑包男双肩包商务笔记本背包男士大容量15.6寸旅行出差休闲书包
 2